# Data Visualisation of capitals_lat_lon.csv

Estimating the latitude and longitude for three points of a triangle that would just cover Africa or come close using shapely and folium

Estimating the boundaries for Kansas and creating geojson file

Generating a random “happiness index” for each state in the USA

In [5]:
import pandas as pd
import io
import folium
from shapely.geometry import Point, Polygon
import geopandas as gpd
import json
import plotly.express as px
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

PART 1

In [3]:
from google.colab import files
uploaded = files.upload()

Saving capitals_lat_lon.csv to capitals_lat_lon.csv


In [8]:
data = pd.read_csv(io.BytesIO(uploaded['capitals_lat_lon.csv']))
print(data)

               Country    Capital   Latitude   Longitude
0             Abkhazia    Sukhumi  43.001525   41.023415
1          Afghanistan      Kabul  34.575503   69.240073
2        Aland Islands  Mariehamn  60.100000   19.933333
3              Albania     Tirana  41.327546   19.818698
4              Algeria    Algiers  36.752887    3.042048
..                 ...        ...        ...         ...
224          Venezuela    Caracas  10.480594  -66.903606
225            Vietnam      Hanoi  21.027764  105.834160
226  Wallis and Futuna   Mata-Utu -13.282509 -176.176447
227             Zambia     Lusaka -15.387526   28.322817
228           Zimbabwe     Harare -17.825166   31.033510

[229 rows x 4 columns]


In [9]:
map_center = [20, 0]
m = folium.Map(location=map_center, zoom_start=2)

for index, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Capital'],
        icon=folium.Icon(icon='cloud')
    ).add_to(m)

m.save('capital_cities_map.html')

PART 2

In [10]:
coordinates = {
    'Rabat': (34.0209, -6.8417),
    'Capetown': (-33.9249, 18.4241),
    'Mogadishu': (2.0469, 45.3182)
}

map_center = [-5, 20]
m = folium.Map(location=map_center, zoom_start=3)
for city, coord in coordinates.items():
    folium.Marker(
        location=coord,
        popup=city
    ).add_to(m)

In [11]:
points = [Point(coord[1], coord[0]) for coord in coordinates.values()]
folium.Polygon(locations=[(p.y, p.x) for p in points], color='blue', fill=True, fill_color='blue', fill_opacity=0.2).add_to(m)

m.save('africa_triangle_map.html')

PART 3

In [12]:
geojson_data = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [-102.0, 39.0],
                [-94.0, 39.0],
                [-94.0, 36.5],
                [-102.0, 36.5],
                [-102.0, 39.0]
            ]
        ]
    },
    "properties": {
        "name": "Kansas"
    }
}

with open('kansas.geojson', 'w') as f:
    json.dump(geojson_data, f)

In [13]:
geojson_data = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [-104.0, 43.0],
                [-95.0, 43.0],
                [-95.0, 40.0],
                [-104.0, 40.0],
                [-104.0, 43.0]
            ]
        ]
    },
    "properties": {
        "name": "Nebraska"
    }
}

with open('nebraska.geojson', 'w') as f:
    json.dump(geojson_data, f)

In [14]:
def plot_geojson(geojson_file, color, label):
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)

    m = folium.Map(location=[41.5, -99.0], zoom_start=6)
    folium.GeoJson(geojson_data, style_function=lambda x: {'fillColor': color, 'color': color}).add_to(m)

    folium.Marker(
        location=[40.5, -96.0],
        icon=None,
        popup=label,
    ).add_to(m)

    return m

In [15]:
if __name__ == "__main__":
    kansas_map = plot_geojson('kansas.geojson', color='blue', label='Kansas')
    nebraska_map = plot_geojson('nebraska.geojson', color='green', label='Nebraska')

    kansas_map.save('kansas_map.html')
    nebraska_map.save('nebraska_map.html')

PART 4

In [16]:
np.random.seed(42)
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

happiness_index = np.random.uniform(0, 100, len(states))

data = pd.DataFrame({'State': states, 'Happiness Index': happiness_index})

In [17]:
fig = px.choropleth(data,
                    locations="State",
                    locationmode='USA-states',
                    color="Happiness Index",
                    color_continuous_scale="Viridis",
                    scope="usa",
                    labels={'Happiness Index': 'Happiness Index'},
                    title="Random Happiness Index by State in the USA")

fig.update_geos(
    visible=False,
    lonaxis_range=[-130, -60],
    projection_scale=7
)
fig.show()

Please zoom out of the choropleth to see the entire map.

PART 5

In [18]:
np.random.seed(42)
data = np.random.exponential(scale=2, size=500)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='histogram'),
    dcc.Slider(
        id='bin-slider',
        min=5,
        max=30,
        step=1,
        value=15,
        marks={i: str(i) for i in range(5, 31, 5)}
    )
])

In [19]:
@app.callback(
    Output('histogram', 'figure'),
    [Input('bin-slider', 'value')]
)
def update_histogram(selected_bins):
    fig = px.histogram(data, nbins=int(selected_bins))
    fig.update_layout(title=f'Histogram (Bins: {selected_bins})', xaxis_title='Value', yaxis_title='Count')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>